In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pd.set_option('display.max_colwidth', None)

/kaggle/input/stanford-rna-3d-folding/sample_submission.csv
/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv
/kaggle/input/stanford-rna-3d-folding/test_sequences.csv
/kaggle/input/stanford-rna-3d-folding/validation_labels.csv
/kaggle/input/stanford-rna-3d-folding/train_labels.csv
/kaggle/input/stanford-rna-3d-folding/train_sequences.csv


## DATA LANDSCAPE

In [2]:
sample = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/sample_submission.csv')
sample

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1107_1,G,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,R1107_2,G,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R1107_3,G,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,R1107_4,G,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,R1107_5,G,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1189_114,U,114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2511,R1189_115,U,115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2512,R1189_116,U,116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2513,R1189_117,U,117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_train_seq =pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')

In [4]:
df_train_seq.head()

,target_id,sequence,temporal_cutoff,description,all_sequences
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus norvegicus (10116)\nGGGUGCUCAGUACGAGAGGAACCGCACCC\n
1,1RNK_A,GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU,1995-02-27,THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES EFFICIENT FRAMESHIFTING IN MOUSE MAMMARY TUMOR VIRUS,>1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU\n
2,1RHT_A,GGGACUGACGAUCACGCAGUCUAU,1995-06-03,"24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE FOR BACTERIOPHAGE R17 (NMR, MINIMIZED AVERAGE STRUCTURE)",>1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*GP*AP*CP*GP*AP*UP*CP*AP*CP*GP*CP*AP*GP*UP*CP*UP*AP*U)-3')|null\nGGGACUGACGAUCACGCAGUCUAU\n
3,1HLX_A,GGGAUAACUUCGGUUGUCCC,1995-09-15,P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID,>1HLX_1|Chain A|RNA (5'-R(*GP*GP*GP*AP*UP*AP*AP*CP*UP*UP*CP*GP*GP*UP*UP*GP*UP*CP*CP*C)-3')|null\nGGGAUAACUUCGGUUGUCCC\n
4,1HMH_E,GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU,1995-12-07,THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RIBOZYME,">1HMH_1|Chains A, C, E|HAMMERHEAD RIBOZYME-RNA STRAND|\nGGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU\n>1HMH_2|Chains B, D, F|HAMMERHEAD RIBOZYME-DNA STRAND|\nACGGTCGGTCGCC\n"


In [5]:
df_train_seq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844 entries, 0 to 843
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   target_id        844 non-null    object
 1   sequence         844 non-null    object
 2   temporal_cutoff  844 non-null    object
 3   description      844 non-null    object
 4   all_sequences    839 non-null    object
dtypes: object(5)
memory usage: 33.1+ KB


In [6]:
df_train_seq.shape

(844, 5)

In [7]:
df_train_seq['sequence'].apply(len).sum()   #this equals shape of train labels

137095

In [8]:
df_train_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv')

In [9]:
df_train_labels.head()

,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793


In [10]:
df_train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137095 entries, 0 to 137094
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ID       137095 non-null  object 
 1   resname  137095 non-null  object 
 2   resid    137095 non-null  int64  
 3   x_1      130950 non-null  float64
 4   y_1      130950 non-null  float64
 5   z_1      130950 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 6.3+ MB


In [11]:
df_train_labels.shape

(137095, 6)

In [12]:
df_train_labels.resid.max()

4298

In [13]:
## Lets separate resID from ID
df_train_labels['ID_lastdig']=pd.to_numeric(df_train_labels['ID'].str.split('_').str[2])

In [14]:
df_train_labels.head()

,ID,resname,resid,x_1,y_1,z_1,ID_lastdig
0,1SCL_A_1,G,1,13.760,-25.974001,0.102,1
1,1SCL_A_2,G,2,9.310,-29.638000,2.669,2
2,1SCL_A_3,G,3,5.529,-27.813000,5.878,3
3,1SCL_A_4,U,4,2.678,-24.900999,9.793,4
4,1SCL_A_5,G,5,1.827,-20.136000,11.793,5


In [15]:
df_train_labels['check']=(df_train_labels['ID_lastdig']==df_train_labels['resid'])

In [16]:
df_train_labels['check'].value_counts()

check
True    137095
Name: count, dtype: int64

In [17]:
df_test_seq=pd.read_csv('/kaggle/input/stanford-rna-3d-folding/test_sequences.csv')

In [18]:
df_test_seq.shape

(12, 5)

In [19]:
df_test_seq

,target_id,sequence,temporal_cutoff,description,all_sequences
0,R1107,GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU,2022-05-28,CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ribozyme,>7QR4_1|Chain A|U1 small nuclear ribonucleoprotein A|Homo sapiens (9606)\nRPNHTIYINNLNEKIKKDELKKSLHAIFSRFGQILDILVSRSLKMRGQAFVIFKEVSSATNALRSMQGFPFYDKPMRIQYAKTDSDIIAKM\n>7QR4_2|Chain B|RNA CPEB3 ribozyme|Homo sapiens (9606)\nGGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU
1,R1108,GGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU,2022-05-27,CPEB3 ribozyme\nChimpanzee\nChimpanzee CPEB3 HDV-like ribozyme,">7QR3_1|Chains A, B|U1 small nuclear ribonucleoprotein A|Homo sapiens (9606)\nRPNHTIYINNLNEKIKKDELKKSLHAIFSRFGQILDILVSRSLKMRGQAFVIFKEVSSATNALRSMQGFPFYDKPMRIQYAKTDSDIIAKM\n>7QR3_2|Chains C, D|chimpanzee CPEB3 ribozyme|Pan troglodytes (9598)\nGGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU"
2,R1116,CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGGGGUUGUACCCACCCCAGAGGCCCACGUGGCGGCUAGUACUCCGGUAUUGCGGUACCCUUGUACGCCUGUUUUAGCCGCGGGUCCAGGGUUCAAGUCCCUGUUCGGGCGCCA,2022-06-04,Cloverleaf RNA\nPoliovirus\nCrystal Structure of Poliovirus (type 1 Mahoney) cloverleaf RNA with tRNA scaffold,">8S95_1|Chain A[auth C]|Lysine tRNA scaffold,Poliovirus cloverleaf RNA|Homo sapiens (9606)\nCGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGGGGUUGUACCCACCCCAGAGGCCCACGUGGCGGCUAGUACUCCGGUAUUGCGGUACCCUUGUACGCCUGUUUUAGCCGCGGGUCCAGGGUUCAAGUCCCUGUUCGGGCGCCA"
3,R1117v2,UUGGGUUCCCUCACCCCAAUCAUAAAAAGG,2022-06-03,PreQ1 class I type III riboswitch\nK. pneumoniae\nAdditional Information: This is a ligand-only target (re-release of R1117 with the corrected SMILES string).\nID Name SMILES Relevant\n001 PRF NCc1c[nH]c2nc(N)[nH]c(=O)c12 Yes\nClass I type III preQ1 riboswitch from E. coli,">8FZA_1|Chains A, B|PreQ1 Riboswitch (30-MER)|Escherichia coli (562)\nUUGGGUUCCCUCACCCCAAUCAUAAAAAGG"
4,R1126,GGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUUACACCAUGCAACGCAGACCUGUAGAUGCCACGCUAGCCGUGGUGAGGGUCGGGUCCAGAUGUCAUUCGACUUUAACGCGCCUAAGCGUUGAAGGCGUGUUAGAGCAGAUAGUUCGCUAUCUGGGGAGCCUGUUCGCAGGCUCAGGAGCCUUCGGGCUCCUAGCGCUAUUACCCCGGACACCACCGGGCAGACAAGUAAUGGUGCUCCUCGAAUGACUUCUGUUGAGUAGAGUGUGGGCUCCGCGGCUAGUGUGCACCUUAGCGGUGAAUGUCUGACACCGUUAAGGUGGUUACUCUUCGGAGUAACGCCGAGAUUCC,2022-06-11,Traptamer\nSynthetic\nAdditional Information: Contains a relevant ion.\nRNA origami 3-helix tile Traptamer,>8TVZ_1|Chain A[auth C]|RNA (363-MER)|synthetic construct (32630)\nGGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUUACACCAUGCAACGCAGACCUGUAGAUGCCACGCUAGCCGUGGUGAGGGUCGGGUCCAGAUGUCAUUCGACUUUAACGCGCCUAAGCGUUGAAGGCGUGUUAGAGCAGAUAGUUCGCUAUCUGGGGAGCCUGUUCGCAGGCUCAGGAGCCUUCGGGCUCCUAGCGCUAUUACCCCGGACACCACCGGGCAGACAAGUAAUGGUGCUCCUCGAAUGACUUCUGUUGAGUAGAGUGUGGGCUCCGCGGCUAGUGUGCACCUUAGCGGUGAAUGUCUGACACCGUUAAGGUGGUUACUCUUCGGAGUAACGCCGAGAUUCC
5,R1128,GGAAUAUCGUCAUGGUGAUUCGUCACCAUGAGGCUAGAUCUCAUAUCUAGCGCUUUCGAGCGCUAGAGUCCUUAUCUAGCCGGUUUAUACUUUCGAGUGUGAACCCGAUAUUCCGCGGAUCACUAUGAGUCGUUCGCGGCUCAUAGUCCGGCUCAAAGGACAUCAUGGCCUGUUCGCAGGUUGUGAUUAUGAGUGAGCCGGGUAAGGCAUACCGUUCGCGGUAUGUCUUACGAUCCGC,2022-06-10,6WJ\nSingle-stranded Paranemic Crossover RNA Triangle (PXT),>8BTZ_1|Chain A|RNA Paranemic croosover triangle (PXT)|synthetic construct (32630)\nGGAAUAUCGUCAUGGUGAUUCGUCACCAUGAGGCUAGAUCUCAUAUCUAGCGCUUUCGAGCGCUAGAGUCCUUAUCUAGCCGGUUUAUACUUUCGAGUGUGAACCCGAUAUUCCGCGGAUCACUAUGAGUCGUUCGCGGCUCAUAGUCCGGCUCAAAGGACAUCAUGGCCUGUUCGCAGGUUGUGAUUAUGAGUGAGCCGGGUAAGGCAUACCGUUCGCGGUAUGUCUUACGAUCCGC
6,R1136,GGAUACGUCUACGCUCAGUGACGGACUCUCUUCGGAGAGUCUGACAUCCGAACCAUACACGGAUGUGCCUCGCCGAACAGUCUACGGCGAGCUUAAGCGCUGGGGACGCCCAACGCAUCACAAAGACUGAGUGAUGAACCAGAAGUAUGGACUGGUUGCGUUGGUGGAGACGGUCGGGUCCAGUUCGCUGUCGAGUAGAGUGUGGGCUCCAUCGACGCCGCUUUAAGGUCCCCAAUCGUGGCGUGUCGGCCUGCUUCGGCAGGCACUGGCGCCGGGACCUUGAAGAGAUGAGAUUUCGAUCUCAUCUUUGGGUGUCUCUGGUGCUUGAGGGCCCUGUGUUCGCACAGGGCCGCUCACUGGGUGUGGACGUAUCC,2022-06-18,Apta-FRET\nAdditional Information: Information about the bound ligand is provided in SMILES section below.\nID Na

In [20]:
# In additional notes date of discovery is more than 2022-07-27

df_train_seq_fltrd=df_train_seq[df_train_seq.temporal_cutoff>'2022-05-27']

In [21]:
df_train_seq_fltrd.shape

(152, 5)

In [22]:
df_val_seq=pd.read_csv('/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv')

In [23]:
df_val_seq.head()

,target_id,sequence,temporal_cutoff,description,all_sequences
0,R1107,GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU,2022-05-28,CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ribozyme,>7QR4_1|Chain A|U1 small nuclear ribonucleoprotein A|Homo sapiens (9606)\nRPNHTIYINNLNEKIKKDELKKSLHAIFSRFGQILDILVSRSLKMRGQAFVIFKEVSSATNALRSMQGFPFYDKPMRIQYAKTDSDIIAKM\n>7QR4_2|Chain B|RNA CPEB3 ribozyme|Homo sapiens (9606)\nGGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU
1,R1108,GGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU,2022-05-27,CPEB3 ribozyme\nChimpanzee\nChimpanzee CPEB3 HDV-like ribozyme,">7QR3_1|Chains A, B|U1 small nuclear ribonucleoprotein A|Homo sapiens (9606)\nRPNHTIYINNLNEKIKKDELKKSLHAIFSRFGQILDILVSRSLKMRGQAFVIFKEVSSATNALRSMQGFPFYDKPMRIQYAKTDSDIIAKM\n>7QR3_2|Chains C, D|chimpanzee CPEB3 ribozyme|Pan troglodytes (9598)\nGGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU"
2,R1116,CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGGGGUUGUACCCACCCCAGAGGCCCACGUGGCGGCUAGUACUCCGGUAUUGCGGUACCCUUGUACGCCUGUUUUAGCCGCGGGUCCAGGGUUCAAGUCCCUGUUCGGGCGCCA,2022-06-04,Cloverleaf RNA\nPoliovirus\nCrystal Structure of Poliovirus (type 1 Mahoney) cloverleaf RNA with tRNA scaffold,">8S95_1|Chain A[auth C]|Lysine tRNA scaffold,Poliovirus cloverleaf RNA|Homo sapiens (9606)\nCGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGGGGUUGUACCCACCCCAGAGGCCCACGUGGCGGCUAGUACUCCGGUAUUGCGGUACCCUUGUACGCCUGUUUUAGCCGCGGGUCCAGGGUUCAAGUCCCUGUUCGGGCGCCA"
3,R1117v2,UUGGGUUCCCUCACCCCAAUCAUAAAAAGG,2022-06-03,PreQ1 class I type III riboswitch\nK. pneumoniae\nAdditional Information: This is a ligand-only target (re-release of R1117 with the corrected SMILES string).\nID Name SMILES Relevant\n001 PRF NCc1c[nH]c2nc(N)[nH]c(=O)c12 Yes\nClass I type III preQ1 riboswitch from E. coli,">8FZA_1|Chains A, B|PreQ1 Riboswitch (30-MER)|Escherichia coli (562)\nUUGGGUUCCCUCACCCCAAUCAUAAAAAGG"
4,R1126,GGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUUACACCAUGCAACGCAGACCUGUAGAUGCCACGCUAGCCGUGGUGAGGGUCGGGUCCAGAUGUCAUUCGACUUUAACGCGCCUAAGCGUUGAAGGCGUGUUAGAGCAGAUAGUUCGCUAUCUGGGGAGCCUGUUCGCAGGCUCAGGAGCCUUCGGGCUCCUAGCGCUAUUACCCCGGACACCACCGGGCAGACAAGUAAUGGUGCUCCUCGAAUGACUUCUGUUGAGUAGAGUGUGGGCUCCGCGGCUAGUGUGCACCUUAGCGGUGAAUGUCUGACACCGUUAAGGUGGUUACUCUUCGGAGUAACGCCGAGAUUCC,2022-06-11,Traptamer\nSynthetic\nAdditional Information: Contains a relevant ion.\nRNA origami 3-helix tile Traptamer,>8TVZ_1|Chain A[auth C]|RNA (363-MER)|synthetic construct (32630)\nGGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUUACACCAUGCAACGCAGACCUGUAGAUGCCACGCUAGCCGUGGUGAGGGUCGGGUCCAGAUGUCAUUCGACUUUAACGCGCCUAAGCGUUGAAGGCGUGUUAGAGCAGAUAGUUCGCUAUCUGGGGAGCCUGUUCGCAGGCUCAGGAGCCUUCGGGCUCCUAGCGCUAUUACCCCGGACACCACCGGGCAGACAAGUAAUGGUGCUCCUCGAAUGACUUCUGUUGAGUAGAGUGUGGGCUCCGCGGCUAGUGUGCACCUUAGCGGUGAAUGUCUGACACCGUUAAGGUGGUUACUCUUCGGAGUAACGCCGAGAUUCC


In [24]:
df_val_seq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   target_id        12 non-null     object
 1   sequence         12 non-null     object
 2   temporal_cutoff  12 non-null     object
 3   description      12 non-null     object
 4   all_sequences    12 non-null     object
dtypes: object(5)
memory usage: 608.0+ bytes


In [25]:
df_val_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/validation_labels.csv')

In [26]:
df_val_labels.shape

(2515, 123)

In [27]:
df_val_labels.head()

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_37,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40
0,R1107_1,G,1,-5.499,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
1,R1107_2,G,2,-5.826,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2,R1107_3,G,3,-5.849,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
3,R1107_4,G,4,-5.784,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
4,R1107_5,G,5,-5.755,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18


## EDA

In [28]:
df_train_seq.isna().sum()

target_id          0
sequence           0
temporal_cutoff    0
description        0
all_sequences      5
dtype: int64

In [29]:
df_train_seq[['target_id', 'sequence', 'description', 'all_sequences']] = \
df_train_seq[['target_id', 'sequence', 'description', 'all_sequences']].astype('str')
df_train_seq['temporal_cutoff']=pd.to_datetime(df_train_seq['temporal_cutoff'])

In [30]:
df_train_labels.isna().mean()*100

ID            0.000000
resname       0.000000
resid         0.000000
x_1           4.482293
y_1           4.482293
z_1           4.482293
ID_lastdig    0.000000
check         0.000000
dtype: float64

In [31]:
df_train_labels.head(1)

,ID,resname,resid,x_1,y_1,z_1,ID_lastdig,check
0,1SCL_A_1,G,1,13.76,-25.974001,0.102,1,True


In [32]:
df_train_labels[['ID', 'resname']]=df_train_labels[['ID', 'resname']].astype('str')
df_train_labels[['resid', 'x_1', 'y_1', 'z_1']] = df_train_labels[['resid', 'x_1', 'y_1', 'z_1']].apply(pd.to_numeric)

In [33]:
from ydata_profiling import ProfileReport

In [34]:
profile = ProfileReport(df_train_seq, explorative=True)
profile.to_file("eda_report_sq_data.html")  # Save report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
profile = ProfileReport(df_train_labels, explorative=True)
profile.to_file("eda_report_lbl_data.html")  # Save report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

* Some interesting facts show up on doing EDA

In [36]:
#Lets merge seq. with labels to see the number of matches we have
# First grouping by first ID part i.e. target id and getting the first C coords 
# Lets get the first part here
def merge_labels_seq(df_labels, df_seq):
    df_labels['targ_ID'] = df_labels['ID'].str.split('_').str[:2].apply('_'.join)
    
    #df_train_labels['targ_ID'].nunique() #=844
    
    df_temp = df_labels[df_labels['resname']=='C'].copy()
    
    df_temp['C_rank']=df_temp.groupby('targ_ID')['resid'].rank()
    
    df_first_C=df_temp[df_temp['C_rank']==1]
    
    list_id_absent=set(df_seq['target_id']).difference(set(df_first_C.targ_ID))
    
    df_1_merge=df_seq.merge(df_first_C.drop(columns=['ID_lastdig', 'check', 'C_rank']),left_on='target_id',right_on='targ_ID',how='left')

    return list_id_absent, df_1_merge

In [37]:
_, df_train = merge_labels_seq(df_train_labels, df_train_seq)

In [38]:
df_train.shape

(844, 12)

In [39]:
df_train.head(1)

,target_id,sequence,temporal_cutoff,description,all_sequences,ID,resname,resid,x_1,y_1,z_1,targ_ID
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus norvegicus (10116)\nGGGUGCUCAGUACGAGAGGAACCGCACCC\n,1SCL_A_6,C,6.0,2.04,-14.908,11.771,1SCL_A


In [40]:
# Lets combine an NLP model with features like sequence, description, all sequence 
# numeric data - temporal cutoff (days till now ), first_C_position and take a SVM kernel here 


import numpy as np
import pandas as pd
import torch
from ribonanzanet import RibonanzaNet

# Load test sequences
test = pd.read_csv("test_sequences.csv")  # Assumed to have columns: ID, resname, resid, sequence

# Initialize the RibonanzaNet model (use pretrained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RibonanzaNet().to(device)
model.eval()

# Function to predict RNA structure using RibonanzaNet
def predict_structure(sequence, num_predictions=5):
    """Generates multiple 3D structure predictions for an RNA sequence."""
    coords = []
    for _ in range(num_predictions):
        with torch.no_grad():
            pred_coords = model.predict(sequence)  # Predict (x, y, z) for each residue
        coords.append(pred_coords.cpu().numpy())  # Convert to NumPy
    return np.stack(coords, axis=-1)  # Shape: (num_residues, 3, 5)

# Generate predictions for each sequence in test set
submission_data = []
for _, row in test.iterrows():
    ID, resname, resid, sequence = row["ID"], row["resname"], row["resid"], row["sequence"]
    
    # Predict 5 different structures
    structure_predictions = predict_structure(sequence)  # Shape: (num_residues, 3, 5)
    
    for i, (x_coords, y_coords, z_coords) in enumerate(structure_predictions):
        row_data = [ID, resname, resid]
        for j in range(5):
            row_data.extend([x_coords[j], y_coords[j], z_coords[j]])  # x_j, y_j, z_j
        submission_data.append(row_data)

# Create DataFrame
columns = ["ID", "resname", "resid"] + [f"x_{i},y_{i},z_{i}" for i in range(1, 6)]
submission_df = pd.DataFrame(submission_data, columns=columns)

# Save submission file
submission_df.to_csv("submission.csv", index=False)

print("Submission file generated: submission.csv")
